# Santa 2021 - The Merry Movie Montage

People seem to be getting in the Christmas spirit earlier and earlier each year. Decorations appear for sale in stores in the fall, Christmas songs are on the radio in October…

The Elves at the North Pole are starting to recognize this, and need to work as fast as possible to launch their latest holiday offering: SantaTV+! A 24/7 streaming television channel where it’s “Always Christmas, All the Time.” To debut their new station, they’ve decided to kick things off with a made-for-television Christmas movie marathon! They’re excited for the premiere of such movies as 🎅, 🤶, 🦌, 🧝, 🎄, 🎁, and 🎀!

But elves know that just as important as the movie themselves is the order they’ll be aired. So the elves have decided the best way to figure out which order is best is to watch all the movies in every possible combination to see which feels the most Christmas-y.

Your job is to help the elves by giving them the shortest viewing schedules that shows them every combination of movies so they can get SantaTV+ live as soon as possible! The elves have formed three movie-watching teams to lighten the load, so every combination must be seen by at least one of their groups. But they’re also pretty sure they want to kick off the movie marathon with the 🎅 and 🤶 movies back-to-back, so be sure that each group has all the combinations that start with those. And finally, the elves have agreed to two sugar breaks, so you’re allowed to give each group up to two 🌟 wildcards, which will play all the movies at once while they’re snacking, which will help speed things along.

They can’t launch SantaTV+ until all the groups have finished watching - so help give them the most efficient schedule to see every Christmas movie combination, and help them get back to making toys!

https://www.kaggle.com/c/santa-2021/overview

In [81]:
import csv
import copy
import numpy as np
import pandas as pd
import os
import itertools

from datetime import datetime

%config Completer.use_jedi = False

## Common functions

In [26]:
def get_perms(movies: list) -> list:
    """Given list of movies, list of permutations"""
    
    perm_raw = list(itertools.permutations(movies))
    perms = []
    for p in perm_raw:
        perm = ""
        for item in p:
            perm += str(item)
        perms.append(perm)
    
    return perms

In [27]:
def get_perm_nos(perms: list) -> list:
    """Given of permutations, return perm numbers"""
    
    perm_nos = []
    for i in range(len(perms)):
        perm_nos.append(i)
    
    return perm_nos

In [34]:
def get_distance_matrix(perms: list, perm_nos: list):
    """Given list of permutations, return distance matrix"""
    
    idx = list(perms)
    cols = list(perms)
    df_dist = pd.DataFrame(columns=cols, index=idx)
    for r in list(df_dist.index):
        for c in list(df_dist.columns):
            if str(r)[1:] == str(c)[:3]:
                df_dist.loc[r, c] = 1
            elif str(r)[2:] == str(c)[:2]:
                df_dist.loc[r, c] = 2
            elif str(r)[3:] == str(c)[:1]:
                df_dist.loc[r, c] = 3
            else:
                df_dist.loc[r, c] = 4

    df_dist_nos = copy.deepcopy(df_dist)
    df_dist_nos.columns = perm_nos
    df_dist_nos.index = perm_nos
    
    return df_dist, df_dist_nos

In [29]:
def get_distance_list(df_dist_nos: pd.DataFrame, dist: int):
    """Given distance matrix (nos), return distance list"""
    
    distance_list = []
    for a in range(1, dist):
        for col in df_dist_nos.columns:
            idx = df_dist_nos.index[df_dist_nos[col] == a].tolist()
            if len(idx) > 0:
                for i in idx:
                    row = [a, i, col]
                    distance_list.append(row)
    return distance_list

## Test code

In [ ]:
movies = ['1', '2', '3', '4']
perms = get_perms(movies)
perm_nos = get_perm_nos(perms)
df_dist, df_dist_nos = get_distance_matrix(perms, perm_nos)
distance_list = get_distance_list(df_dist_nos, len(movies)+1)

In [ ]:
mandatory_perms = [p for p in perms if p[0]=='0' and p[1]=='1']
mandatory_perm_nos = []
for p in mandatory_perms:
    mandatory_perm_nos.append(perm_nos[perms.index(p)])

In [36]:
distance_list_mandatory = [d for d in distance_list if d[2] in mandatory_perm_nos]
distance_list_nonmandatory = [d for d in distance_list if d[2] not in mandatory_perm_nos]

## Main code

In [ ]:
path = '../data/'
with open(path + 'permutations.csv', newline='', encoding="utf-8") as f:
    reader = csv.reader(f)
    p = list(reader)

p = p[1:]
perms = [a[0] for a in p]
perm_nos = []
i = 0
for i in range(len(perms)):
    perm_nos.append(i)

with open(path + 'distance_matrix.csv', newline='', encoding="utf-8") as f:
    reader = csv.reader(f)
    d = list(reader)

df_dist = pd.read_csv(path + 'distance_matrix.csv')
df_dist.set_index('Permutation', drop=True, inplace=True)

df_dist_nos = copy.deepcopy(df_dist)
cols = perm_nos
idx = perm_nos
df_dist_nos.columns = cols
df_dist_nos.index = cols

In [43]:
distance_list = []
for a in range(1, 8):
    for col in df_dist_nos.columns:
        idx = df_dist_nos.index[df_dist_nos[col] == a].tolist()
        if len(idx) > 0:
            for i in idx:
                row = [a, i, col]
                distance_list.append(row)

In [ ]:
mandatory_perms = [p for p in perms if p[0]=='🎅' and p[1]=='🤶']
mandatory_perm_nos = []
for p in mandatory_perms:
    mandatory_perm_nos.append(perm_nos[perms.index(p)])

In [45]:
distance_list_mandatory = [d for d in distance_list if d[2] in mandatory_perm_nos]
distance_list_nonmandatory = [d for d in distance_list if d[2] not in mandatory_perm_nos]

## Common

In [115]:
# initialize
num_strs = 3
strs = []
rem_mand_perms = [] # remaining list of mandatory perms
str_dists = [] # list of strings

rem_perms = [p for p in perm_nos if p not in mandatory_perm_nos]

for i in range(num_strs):
    new_str = [mandatory_perm_nos[i]]
    new_mand_perms = copy.deepcopy(mandatory_perm_nos)
    new_mand_perms.remove(mandatory_perm_nos[i])
    
    rem_mand_perms.append(new_mand_perms)
    strs.append([new_str])
    str_dists.append([])

**Algorithm**
1. 
  - for each string  
  - check if any mandatory perms can be found with dist 1  
  - if yes, add perm, add distance, reset dist and restart from 1
  - if no, continue
2. 
  - identify string with shortest distance
  - check if any non-mandatory perms can be found with current dist
  - if yes, add perm, add distance, reset dist and restart from 1
  - if no, increment dist, restart from 1

**Notes**
1. Any time any perm is added, dist is reset and loop is restarted
2. Non-mandatory perms are added to string with smallest distance

In [116]:
dist = 1
print(len(rem_perm_nos))
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)
        
ctr = 1
max_ctr = -1
while len(rem_perms) > 0 and (ctr <= max_ctr or max_ctr == -1):
    ctr += 1
    if len(rem_perms) % 100 == 0:
        print(len(rem_perms))
        now = datetime.now()

        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time)
        
    # attempt to find d1 remaining mandatory perms in each list
    loop1 = True
    while loop1:
        loop1 = False
        for i in range(num_strs):        
            trail = strs[i][0][-1]
            list_match = [p for p in distance_list_mandatory if p[0]==1 and p[1] == trail and p[2] in rem_mand_perms[i]]
            
            if len(list_match) > 0:
                strs[i][0].append(list_match[0][2])
                rem_mand_perms[i].remove(list_match[0][2])
                trail = strs[i][0][-1]
                str_dists[i].append(dist)
                loop1 = True
                if dist > 1:
                    dist = 1
            
        
    # get string with shortest length
    length = 10000000
    shortest_i = -1
    for i in range(num_strs):
        if len(str_dists[i]) < length:
            length = len(str_dists[i])
            shortest_i = i
    i = shortest_i
    
    # check if any perms can be found with dist
    trail = strs[i][0][-1]
    list_match = [p for p in distance_list_nonmandatory if p[0]==dist and p[1] == trail and p[2] in rem_perms]
    if list_match:
        strs[i][0].append(list_match[0][2])
        rem_perms.remove(list_match[0][2])
        str_dists[i].append(dist)
        if dist > 1:
            dist = 1
        continue
    else:
        dist += 1
               
now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

4920
Start Time = 18:51:43
4900
Current Time = 18:52:18
4900
Current Time = 18:52:20
4800
Current Time = 18:55:50
4700
Current Time = 18:59:37
4700
Current Time = 18:59:39
4600
Current Time = 19:03:38
4500
Current Time = 19:07:30
4400
Current Time = 19:11:32
4300
Current Time = 19:15:50
4200
Current Time = 19:20:13
4100
Current Time = 19:24:37
4000
Current Time = 19:28:21
3900
Current Time = 19:31:46
3800
Current Time = 19:35:16
3700
Current Time = 19:39:19
3600
Current Time = 19:42:49
3600
Current Time = 19:42:50
3500
Current Time = 19:46:09
3400
Current Time = 19:49:33
3300
Current Time = 19:52:56
3200
Current Time = 19:56:32
3100
Current Time = 19:59:51
3100
Current Time = 19:59:53
3100
Current Time = 19:59:55
3000
Current Time = 20:03:10
2900
Current Time = 20:06:40
2800
Current Time = 20:10:11
2700
Current Time = 20:13:32
2600
Current Time = 20:16:57
2500
Current Time = 20:20:21
2400
Current Time = 20:23:48
2300
Current Time = 20:27:12
2200
Current Time = 20:30:33
2100
Current Tim

In [145]:
p_nm = [p for p in perm_nos if p not in mandatory_perm_nos]
p_nm_np = [p for p in p_nm if (p not in strs[0][0]) and (p not in strs[1][0]) and (p not in strs[2][0])]
p_m = [p for p in perm_nos if p in mandatory_perm_nos]
p_m_np1 = [p for p in p_m if p not in strs[0][0]]
p_m_np2 = [p for p in p_m if p not in strs[1][0]]
p_m_np3 = [p for p in p_m if p not in strs[2][0]]

print('string 1 length: %d' % (len(str_dists[0])))
print('string 2 length: %d' % (len(str_dists[1])))
print('string 3 length: %d' % (len(str_dists[2])))
print('total distance: %d' % (len(str_dists[0]) + len(str_dists[1]) + len(str_dists[2])))

print('nonmandatory perms %d' % (len(p_nm)))
print('nonmandatory perms not in any string: %d' % (len(p_nm_np)))
print('mandatory perms %d' % (len(p_m)))
print('mandatory perms not in s1: %d' % (len(p_m_np1)))
print('mandatory perms not in s2: %d' % (len(p_m_np2)))
print('mandatory perms not in s3: %d' % (len(p_m_np3)))

string 1 length: 1679
string 2 length: 1679
string 3 length: 1679
total distance: 5037
nonmandatory perms 4920
nonmandatory perms not in any string: 0
mandatory perms 120
mandatory perms not in s1: 83
mandatory perms not in s2: 79
mandatory perms not in s3: 78
